In [1]:
import GPy
import gp_estimator
import numpy as np

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
x = np.array([[0.0055, 0.    , 2.0611, 2.1779, 2.049 , 2.1829],
 [0.0162, 0.0008, 2.404 , 2.4205, 2.3808, 2.438 ],
 [0.0293, 0.0023, 3.0397, 2.9771, 3.0244, 2.9875],
 [0.0504, 0.0048, 3.6016, 3.5267, 3.6161, 3.5082],
 [0.0793, 0.0088, 3.9944, 3.9703, 4.0511, 3.9107],
 [0.1071, 0.0141, 4.2249, 4.2913, 4.3218, 4.1928],
 [0.12  , 0.0201, 4.3301, 4.4981, 4.4512, 4.3768],])
y = np.array([[ 0.0591],
 [ 0.0655],
 [ 0.0651],
 [ 0.0469],
 [ 0.0204],
 [-0.0319],
 [-0.0498],])
x_new = np.array([[ 0.029 ,  0.0304,  2.0851,  2.1765,  2.0631,  2.1932],
 [ 0.0247,  0.0317,  2.3887,  2.3709,  2.3523,  2.4034],
 [ 0.0228,  0.0328,  2.9894,  2.8823,  2.9627,  2.9058],
 [ 0.0274,  0.0342,  3.5229,  3.4013,  3.5312,  3.3905],
 [ 0.0411,  0.0362,  3.8938,  3.831 ,  3.951 ,  3.7719],
 [ 0.057 ,  0.0391,  4.1085,  4.1521,  4.2117,  4.0479],
 [ 0.0608,  0.0421,  4.2044,  4.3692,  4.3341,  4.2392],
 [ 0.0373,  0.044 ,  4.2223,  4.4921,  4.3478,  4.3671],
 [-0.0236,  0.0428,  4.1963,  4.5302,  4.2815,  4.4455],
 [-0.1229,  0.0367,  4.1489,  4.4901,  4.1599,  4.4795],
 [-0.2518,  0.0241,  4.09  ,  4.3761,  4.001 ,  4.4654],
 [ 0.033 ,  0.0273,  2.0986,  2.2043,  2.0896,  2.2101],])

In [2]:
GP = gp_estimator.GP_estimator()

In [7]:
%timeit mean, var = GP.predict_accel(x, y, x_new, 0, True)

2.34 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
